**Goal**: prototype the AST detectors to be used in the flake8 plugin.

# Flake8 Prototypes AST Detectors

In [1]:
import project_path
from rdlib import *  # this imports the base libraries

In [7]:
%load_ext autoreload
%autoreload 2

In [12]:
PATH_DATASET = '../data/04_program_under_study/exp_v01_program_dataset.csv'
PATH_EXAMPLE = '../data/flake8_prototypes/01_measure_all_bad.py'

In [13]:
df = pd.read_csv(PATH_DATASET)
df.head()

,download_url,html_url,name,path,repository_description,repository_fork,repository_name,repository_owner_url,repository_url,score,hash,filename,extension,filestem,unique_id,output_file_name,file_content,file_content_code_only,parsable,loc
0,https://raw.githubusercontent.com/CQCL/tket_be...,https://github.com/CQCL/tket_benchmarking/blob...,chemistry_set_generator.py,arxiv-1902-08091/ChemistrySet/chemistry_set_ge...,Scripts and circuits for benchmarking tket,False,CQCL/tket_benchmarking,https://github.com/CQCL/tket_benchmarking,https://github.com/CQCL/tket_benchmarking,1.0,1b89358c751ae3768afa2bfe3e1e02fd599c6cd29a153b...,chemistry_set_generator.py,py,chemistry_set_generator,chemistry_set_generator_3e40eb,chemistry_set_generator_3e40eb.py,# Script for generating the example circuits i...,from qiskit_chemistry.drivers.pyscfd import Py...,True,38
1,https://raw.githubusercontent.com/dlyongemallo...,https://github.com/dlyongemallo/quantum-comput...,teleport.py,qiskit/teleport.py,Tools for quantum computation,False,dlyongemallo/quantum-computation,https://github.com/dlyongemallo/quantum-comput...,https://github.com/dlyongemallo/quantum-comput...,1.0,20f0394d9296d034bdbe4e74bcf87c645b359c48e1eeae...,teleport.py,py,teleport,teleport_6c481d,teleport_6c481d.py,"#!/usr/bin/env python3\n\n""""""Quantum teleporta...","from qiskit import QuantumCircuit, QuantumRegi...",True,65
2,https://raw.githubusercontent.com/mtreinish/vf...,https://github.com/mtreinish/vf2_partial_layou...,plugin.py,vf2_partial_layout/plugin.py,An experimental Layout Pass for Qiskit's compiler,False,mtreinish/vf2_partial_layout,https://github.com/mtreinish/vf2_partial_layout,https://github.com/mtreinish/vf2_partial_layout,1.0,1c3c584181c2e1666f7e9513bad937684025d4700a22ac...,plugin.py,py,plugin,plugin_1c8fd5,plugin_1c8fd5.py,# This code is part of Qiskit.\n#\n# (C) Copyr...,from qiskit.transpiler import PassManager\nfro...,True,44
3,https://raw.githubusercontent.com/epelofske/qu...,https://github.com/epelofske/quantum_optimizat...,ibmqx_ising_qubo_qaoa.py,utils/ibmqx_ising_qubo_qaoa.py,NaN,False,epelofske/quantum_optimization,https://github.com/epelofske/quantum_optimization,https://github.com/epelofske/quantum_optimization,1.0,11056e3a8afad9f9ed98f7e030f8b32e4ffed28f75f1b1...,ibmqx_ising_qubo_qaoa.py,py,ibmqx_ising_qubo_qaoa,ibmqx_ising_qubo_qaoa_26bcbd,ibmqx_ising_qubo_qaoa_26bcbd.py,from collections import OrderedDict\nimport nu...,from collections import OrderedDict\nimport nu...,True,85
4,https://raw.githubusercontent.com/jcylim/Qiski...,https://github.com/jcylim/QiskitProject/blob/9...,entanglement.py,qiskit/quantum information science/entanglemen...,Personal project to play with and explore the ...,False,jcylim/QiskitProject,https://github.com/jcylim/QiskitProject,https://github.com/jcylim/QiskitProject,1.0,7be0a9977abae76679e928987cee7fa69cbc60c128bd3c...,entanglement.py,py,entanglement,entanglement_544985,entanglement_544985.py,# useful additional packages\nimport matplotli...,import matplotlib.pyplot as plt\nimport numpy ...,True,68


In [20]:
import ast

def get_ast(path):
    with open(path, 'r') as f:
        code = f.read()
    return ast.parse(code)

def prettify(ast_tree_str, indent=4):
    ret = []
    stack = []
    in_string = False
    curr_indent = 0

    for i in range(len(ast_tree_str)):
        char = ast_tree_str[i]
        if in_string and char != '\'' and char != '"':
            ret.append(char)
        elif char == '(' or char == '[':
            ret.append(char)

            if i < len(ast_tree_str) - 1:
                next_char = ast_tree_str[i+1]
                if next_char == ')' or next_char == ']':
                    curr_indent += indent
                    stack.append(char)
                    continue

            print(''.join(ret))
            ret.clear()

            curr_indent += indent
            ret.append(' ' * curr_indent)
            stack.append(char)
        elif char == ',':
            ret.append(char)

            print(''.join(ret))
            ret.clear()
            ret.append(' ' * curr_indent)
        elif char == ')' or char == ']':
            ret.append(char)
            curr_indent -= indent
            stack.pop()
        elif char == '\'' or char == '"':

            if (len(ret) > 0 and ret[-1] == '\\') or (in_string and stack[-1] != char):
                ret.append(char)
                continue

            if len(stack) > 0 and stack[-1] == char:
                ret.append(char)
                in_string = False
                stack.pop()
                continue

            in_string = True
            ret.append(char)
            stack.append(char)
        elif char == ' ':
            pass
        else:
            ret.append(char)

    print(''.join(ret))

tree = get_ast(PATH_EXAMPLE)
prettify(ast.dump(tree))

Module(
    body=[
        ImportFrom(
            module='qiskit',
            names=[
                alias(
                    name='QuantumCircuit',
                    asname=None),
                alias(
                    name='QuantumRegister',
                    asname=None),
                alias(
                    name='ClassicalRegister',
                    asname=None)],
            level=0),
        ImportFrom(
            module='qiskit',
            names=[
                alias(
                    name='execute',
                    asname=None),
                alias(
                    name='Aer',
                    asname=None)],
            level=0),
        Assign(
            targets=[
                Name(
                    id='q',
                    ctx=Store())],
            value=Call(
                func=Name(
                    id='QuantumRegister',
                    ctx=Load()),
                args=[
                    Constant(
         

In [33]:
class CircuitVisitor(ast.NodeVisitor):

    # def visit_Expr(self, node: ast.Expr) -> Any:
    #     print('Expr', node)
    #     return self.generic_visit(node)

    def visit_Call(self, node: ast.Call) -> Any:
        sub_node = node.func
        if isinstance(sub_node, ast.Attribute):
            print('Call (method)', sub_node.attr)
            if sub_node.attr == 'measure_all':
                # add to the error list
                print("Bug!")
        elif isinstance(sub_node, ast.Name):
            print('Call', sub_node.id)
        return self.generic_visit(node)

In [34]:
tree.body

In [35]:
circVis = CircuitVisitor()
circVis.visit(tree)

Call QuantumRegister
Call ClassicalRegister
Call QuantumCircuit
Call (method) h
Call (method) cx
Call (method) measure_all
Call execute
Call (method) get_backend
Call (method) result
Call (method) get_counts
Call print


In [7]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute, Aer
qreg = QuantumRegister(3)
creg = ClassicalRegister(2)  # BUG
circuit = QuantumCircuit(creg, qreg)
circuit.h(0)
circuit.cx(0, 1)
circuit.cx(1, 2)
circuit.measure(qreg, creg)  # BUG
circuit.draw()

CircuitError: 'register size error'